In [6]:
import fsspec

import rucio.client

In [3]:
%run utils_new.py

In [5]:
with open("x509", "rb") as f:
    setup_rucio_and_proxy(x509_data=f.read())

In [15]:
rucio_client = rucio.client.Client(ca_cert="cern_bundle.pem")

In [28]:
rse = "GOOGLE-EU_DATADISK"
filename = "DAOD_PHYSLITE.33080408._000001.pool.root.1"

In [53]:
url = get_signed_url("mc21_13p6TeV", filename, rse=rse, ca_cert="cern_bundle.pem")

In [31]:
fo = fsspec.open(url)

In [37]:
with fo as f:
    data = f.read(10)

In [38]:
data

b'root\x00\x00\xf4\x90\x00\x00'

In [41]:
f = fo.open()

In [47]:
f.fs.cat_ranges([".", "."], [0, 10], [10, 20])

[<InvalidURL .>, <InvalidURL .>]

In [51]:
fs = fsspec.filesystem("http")

In [107]:
fs.client_kwargs

{}

In [54]:
fs.cat_ranges([url, url], [0, 10], [10, 20])

[b'root\x00\x00\xf4\x90\x00\x00', b'\x00d\x13\xcf\xdb\xa9\x134\xe0\x9e']

In [55]:
import uproot

In [57]:
fs.cat_file(url, 0, 10)

b'root\x00\x00\xf4\x90\x00\x00'

In [58]:
import uproot

In [149]:
class FSSpecHttpSource(uproot.source.chunk.Source):
    def __init__(self, file_path, **options):
        self._file_path = file_path
        # will by default use aiohttp with TCPConnection pool of size 100
        # could also play with block_size and cache_type
        self._fs = fsspec.filesystem("http", block_size=0, cache_type="none")
        
    @property
    def closed(self):
        return False # we're opening all the time?
    
    def _make_chunk(self, start, stop, content):
        future = uproot.source.futures.TrivialFuture(content)
        return uproot.source.chunk.Chunk(self, start, stop, future)
    
    def chunk(self, start, stop):
        return self._make_chunk(start, stop, self._fs.cat_file(self._file_path, start, stop))
    
    def chunks(self, ranges, notifications):
        starts = []
        stops = []
        for start, stop in ranges:
            starts.append(start)
            stops.append(stop)
        contents = self._fs.cat_ranges(
            [self._file_path] * len(ranges), starts, stops
        )
        chunks = []
        for content, (start, stop) in zip(contents, ranges):
            chunk = self._make_chunk(start, stop, content)
            if notifications is not None:
                notifications.put(chunk)
            chunks.append(chunk)
        return chunks

In [150]:
f = uproot.open(url,
                http_handler=FSSpecHttpSource
               )

In [151]:
#f["CollectionTree"]["AnalysisJetsAuxDyn.pt"].array()

In [152]:
tree = f["CollectionTree"]

In [153]:
import re

In [154]:
filter_typename="/((std::)*vector<){0,1}(u*int|float|double)/"

In [155]:
#tree.show(filter_typename=filter_typename, filter_name="AnalysisElectrons*")

In [157]:
%%time
arrays = tree.arrays(filter_typename=filter_typename, filter_name="AnalysisElectrons*")

CPU times: user 1.54 s, sys: 45.8 ms, total: 1.58 s
Wall time: 1.77 s


In [110]:
arrays.nbytes / 1024 / 1024

305.6893033981323

In [158]:
arrays

<Array [{...}, {...}, {...}, ..., {...}, {...}] type='20000 * {"AnalysisEle...'>